In [1]:
#training with tiny shakespeare dataset

with open('input.txt', 'r') as f:
    text = f.read()
data = text[:10000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [2]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


In [3]:
import torch
buf = torch.tensor(tokens[:24])
x = buf.view(4,6)
print(x)

# 25 sees 5962, 22307 and tries to predict 198
# 13 doesn't have a next one tho :(



tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])


In [4]:
#better
buf = torch.tensor(tokens[:24+1])
x = buf[:-1].view(4,6) # (Batch, Time); 
# time = context length, batch = number of "trials" ran at the same time
y = buf[1:].view(4,6)
print(x)
print(y)
# yay :) 

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


In [7]:
# standard deviation grows inside the residual stream
x = torch.zeros(768)
n = 100 # e.g. 100 layers
for i in range(n):
    x += n**-0.5*torch.randn(768)
print(x.std())
# in c_proj


# reason why we scale by 1/sqrt(n) is so that our growth is controlled
# and that our residual stream is independent of the number of layers

tensor(0.9998)
